# Section 1: Load Model

In [ ]:
# For dataset
import pandas as pd
import json
import os
import ast
import re
import numpy as np
from datasets import Dataset
import matplotlib.pyplot as plt


import torch
from time import time
 
import wandb
# Set seed
import pickle
set_seed(42)
import seaborn as sns
from copy import deepcopy
from tqdm import tqdm
PATH = "0_Thesis/4_Multi/"

In [2]:
if os.getcwd() == '/root':
    new_path = "/root/0_Thesis/0_final/"
    os.chdir(new_path)
else:
    os.chdir("..") 
print(os.getcwd())

/root/0_Thesis/0_final


In [6]:
model_id = "unsloth/Llama-3.2-1B-Instruct-bnb-4bit"
# model_id ="unsloth/Qwen2.5-14B-Instruct-bnb-4bit"
# model_id = "unsloth/gemma-2-9b-it-bnb-4bit"

In [20]:
df = pd.read_csv("lgb/lgb_with_llm_probs_multi.csv")

In [21]:
df.shape

(44775, 14)

In [22]:
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [23]:
mstral7b = "unsloth/mistral-7b-instruct-v0.3-bnb-4bit"
# llama8b = "unsloth/Meta-Llama-3.1-8B-Instruct-bnb-4bit"
gemma9b = "unsloth/gemma-2-9b-it-bnb-4bit"
qwen14b = "unsloth/Qwen2.5-14B-Instruct-bnb-4bit"
columns = [mstral7b, gemma9b, qwen14b]

In [24]:
df['label_id'].value_counts()

label_id
2    24611
3    11936
1     8228
Name: count, dtype: int64

In [ ]:
mstral7b = "unsloth/mistral-7b-instruct-v0.3-bnb-4bit"
# llama8b = "unsloth/Meta-Llama-3.1-8B-Instruct-bnb-4bit"
gemma9b = "unsloth/gemma-2-9b-it-bnb-4bit"
qwen14b = "unsloth/Qwen2.5-14B-Instruct-bnb-4bit"
columns = [mstral7b, gemma9b, qwen14b]
all_cols = []
for col in columns:
    all_cols.append(col + "_label_1")
    all_cols.append(col + "_label_2")
    all_cols.append(col + "_label_3")


X = df[all_cols]
y = df['label_id']  - 1

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.1, random_state=42)

train_data = lgb.Dataset(X_train, label=y_train)
val_data = lgb.Dataset(X_val, label=y_val, reference=train_data)

# Set LightGBM parameters
params = {
    'objective': 'multiclass',  
    'num_class': len(y.unique()), 
    'metric': 'multi_logloss', 
    'boosting_type': 'gbdt', 
    'num_leaves': 34, 
    'learning_rate': 0.05, 
    'feature_fraction': 0.9, 
    'bagging_fraction': 0.8,  
    'bagging_freq': 5,
    'verbose': -1, 
}

model = lgb.train(
    params,
    train_data,
    valid_sets=[val_data],
    num_boost_round=1000,

)

val_preds = model.predict(X_val, num_iteration=model.best_iteration)
val_preds_binary = np.round(val_preds) 

# Evaluate the meta-learner
# accuracy = accuracy_score(y_val, val_preds_binary)
# accuracy

In [ ]:
val_preds = [list(x).index(max(x)) for x in val_preds]
accuracy = accuracy_score(y_val, val_preds)
accuracy

0.719740955783832